In [29]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from sklearn.metrics import classification_report, confusion_matrix

In [30]:
# Paths to dataset
train_path = r"D:\PFE\Datasets\ForClassification\wdd2017\Training"
val_path = r"D:\PFE\Datasets\ForClassification\wdd2017\Validation"
test_path = r"D:\PFE\Datasets\ForClassification\wdd2017\Testing"

# Parameters
img_height, img_width = 224, 224
batch_size = 32
num_classes = len(os.listdir(train_path))  # Number of classes based on folder structure
initial_lr = 0.0001
epochs = 40

In [31]:
# Parameters
img_height, img_width = 224, 224
batch_size = 32
num_classes = len(os.listdir(train_path))  # Number of classes based on folder structure
initial_lr = 0.0001
epochs = 40

# Advanced Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2]
)


In [32]:

val_test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 3841 images belonging to 5 classes.
Found 987 images belonging to 5 classes.
Found 693 images belonging to 5 classes.


In [33]:
# Load Pretrained Model (VGG16)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze initial layers to retain pretrained features
for layer in base_model.layers[:-4]:  # Fine-tune last 4 layers
    layer.trainable = False

# Build Model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_4           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 512)                 │         262,6

 Total params: 14,979,909 (57.14 MB)

 Trainable params: 7,344,645 (28.02 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [34]:
# Compile Model with Metrics
model.compile(optimizer=Adam(learning_rate=initial_lr),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [36]:
# Callbacks for Optimization
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# Learning Rate Scheduler
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return float(lr)
    else:
        return float(lr * tf.math.exp(-0.1))

lr_schedule = LearningRateScheduler(lr_scheduler)

# Training the Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr, checkpoint, lr_schedule],
    verbose=1
)

# Load Best Model
model = tf.keras.models.load_model('best_model.keras')


Epoch 1/40
121/121 ━━━━━━━━━━━━━━━━━━━━ 1664s 14s/step - accuracy: 0.5873 - loss: 1.0661 - precision_4: 0.7638 - recall_4: 0.4117 - val_accuracy: 0.8521 - val_loss: 0.5287 - val_precision_4: 0.8636 - val_recall_4: 0.8338 - learning_rate: 1.0000e-04
Epoch 2/40
121/121 ━━━━━━━━━━━━━━━━━━━━ 1652s 14s/step - accuracy: 0.8473 - loss: 0.4374 - precision_4: 0.8817 - recall_4: 0.8088 - val_accuracy: 0.8825 - val_loss: 0.5286 - val_precision_4: 0.8950 - val_recall_4: 0.8723 - learning_rate: 1.0000e-04
Epoch 3/40
121/121 ━━━━━━━━━━━━━━━━━━━━ 1536s 13s/step - accuracy: 0.8982 - loss: 0.2968 - precision_4: 0.9252 - recall_4: 0.8782 - val_accuracy: 0.9129 - val_loss: 0.3976 - val_precision_4: 0.9221 - val_recall_4: 0.9108 - learning_rate: 1.0000e-04
Epoch 4/40
121/121 ━━━━━━━━━━━━━━━━━━━━ 1898s 16s/step - accuracy: 0.9106 - loss: 0.2717 - precision_4: 0.9249 - recall_4: 0.8863 - val_accuracy: 0.9058 - val_loss: 0.4990 - val_precision_4: 0.9073 - val_recall_4: 0.9027 - learning_rate: 1.0000e-04
Epoc

In [50]:
print("no way")

no way


In [45]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Evaluate on Test Set
y_pred_probs = model.predict(test_generator)  # Predicted probabilities
y_pred_classes = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class predictions
y_true = test_generator.classes  # True class labels from the generator

# Calculate test accuracy
test_accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Generate Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


C:\Users\DELL\Desktop\PFETesting\paper1\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 81s 14s/step
Test Accuracy: 57.14%

Classification Report:
              precision    recall  f1-score   support

  Brown_Rust       0.92      0.97      0.95        36
     Healthy       0.18      0.67      0.28        15
  Loose_Smut       0.82      1.00      0.90        33
    Septoria       0.47      0.26      0.34        34
 Yellow_Rust       1.00      0.03      0.05        36

    accuracy                           0.57       154
   macro avg       0.68      0.59      0.51       154
weighted avg       0.75      0.57      0.53       154


Confusion Matrix:
[[35  0  1  0  0]
 [ 0 10  5  0  0]
 [ 0  0 33  0  0]
 [ 0 25  0  9  0]
 [ 3 21  1 10  1]]


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Evaluate on Test Set
y_pred_probs = model.predict(test_generator)  # Predicted probabilities
y_pred_classes = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class predictions
y_true = test_generator.classes  # True class labels from the generator

# Calculate test accuracy
test_accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Generate Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


In [42]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Evaluate on Test Set
y_pred_probs = model.predict(test_generator)  # Predicted probabilities
y_pred_classes = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class predictions
y_true = test_generator.classes  # True class labels from the generator

# Calculate test accuracy
test_accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Generate Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


22/22 ━━━━━━━━━━━━━━━━━━━━ 283s 12s/step
Test Accuracy: 47.91%

Classification Report:
              precision    recall  f1-score   support

  Brown_Rust       0.75      0.25      0.37       166
     Healthy       0.49      0.92      0.64       202
  Loose_Smut       0.75      0.96      0.84        96
    Septoria       0.07      0.26      0.11        34
 Yellow_Rust       0.83      0.03      0.05       195

    accuracy                           0.48       693
   macro avg       0.58      0.48      0.40       693
weighted avg       0.66      0.48      0.41       693


Confusion Matrix:
[[ 41  94   3  27   1]
 [  1 185  16   0   0]
 [  0   3  92   1   0]
 [  0  25   0   9   0]
 [ 13  72  12  93   5]]


In [44]:
test_path = r"D:\PFE\Datasets\Testing"
test_generator = val_test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 154 images belonging to 5 classes.


In [48]:
test_path = r"D:\PFE\D2\Testing"
test_generator = val_test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 52 images belonging to 5 classes.
